In [1]:
import numpy as np
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings

In [2]:
llm = OllamaLLM(model="llama3.2", temperature=1)
response = llm.invoke("hi, do u love cats")
print(response)

Hello! I don't have personal feelings or emotions like humans do, but I can certainly chat with you about cats if you'd like! Many people adore cats and their unique personalities, independence, and adorable antics. What's your relationship like with cats? Do you have a feline friend at home?


In [3]:
embeddings = OllamaEmbeddings(model="all-minilm:33m")

texts = [
    "Что такое LangChain?",
    "Как использовать Ollama для создания эмбеддингов?",
    "Какие модели поддерживает Ollama?"
]

vectors = embeddings.embed_documents(texts)

vectors = np.array(vectors)
new_matr = vectors/np.linalg.norm(vectors, axis=1, keepdims=True)
new_matr @ new_matr.T

array([[1.        , 0.19043786, 0.27984475],
       [0.19043786, 1.        , 0.68144236],
       [0.27984475, 0.68144236, 1.        ]])

In [4]:
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.config import Configure
from weaviate.classes.query import MetadataQuery
import requests, json, os

In [5]:
client = weaviate.connect_to_local()
print(client.is_ready())

True


In [6]:
client.collections.delete_all()

In [7]:
questions = client.collections.create(
    name="Question",
    #vectorizer_config=Configure.Vectorizer.text2vec_ollama(     # Configure the Ollama embedding integration
    #    api_endpoint="http://ollama:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
    #    model="all-minilm:33m",                              # The model to use
    #),
    vectorizer_config =[
        Configure.NamedVectors.text2vec_ollama(
            name="title_vector",
            source_properties=["answer"],
            api_endpoint="http://ollama:11434",  # If using Docker, use this to contact your local Ollama instance
            model="all-minilm:33m",  # The model to use, e.g. "nomic-embed-text"
    )
    ],
    generative_config=Configure.Generative.ollama(              # Configure the Ollama generative integration
        api_endpoint="http://ollama:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
        model="llama3.2",                                       # The model to use
    )
)

In [9]:
resp = requests.get(
    "https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json"
)
data = json.loads(resp.text)

questions = client.collections.get("Question")

with questions.batch.dynamic() as batch:
    for d in data:
        batch.add_object({
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        })

In [16]:
questions = client.collections.get("Question")

response = questions.query.near_text(
    query="mammal",
    limit=2
)

for obj in response.objects:
    print(json.dumps(obj.properties, indent=2))

{
  "answer": "species",
  "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  "category": "SCIENCE"
}
{
  "answer": "Elephant",
  "question": "It's the only living mammal in the order Proboseidea",
  "category": "ANIMALS"
}


In [17]:
questions = client.collections.get("Question")

response = questions.generate.near_text(
    query="species",
    limit=2,
    grouped_task="Write a tweet with emojis about these facts."
)

print(response.generated)  # Inspect the generated text

"🌿🦌 Did you know? 🤔 The Gunnison sage grouse is a unique species in its own right! 🐓 And get ready for some serious horns: the eland in Africa can weigh up to 1 ton, making it the largest of its kind! 😮 #WildlifeFacts #Science #AnimalLovers"


In [18]:
client.close()